<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/LAB03_Pr4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
# Importación de librerías a utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from numpy.core.function_base import linspace
from math import floor
from math import ceil
from numpy.ma.core import size

from sklearn import tree
from sklearn.datasets import load_iris
import graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [47]:
# Lectura de la base de datos
filename = 'lluvias.csv'
data = pd.read_csv(filename, sep = ',')

# Revisión de los datos
print(data.shape)

data.info()
data.head(10)

(25551, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25551 entries, 0 to 25550
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DATE    25551 non-null  object 
 1   PRCP    25548 non-null  float64
 2   TMAX    25551 non-null  int64  
 3   TMIN    25551 non-null  int64  
 4   RAIN    25548 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 998.2+ KB


,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,51,42,True
1,1948-01-02,0.59,45,36,True
2,1948-01-03,0.42,45,35,True
3,1948-01-04,0.31,45,34,True
4,1948-01-05,0.17,45,32,True
5,1948-01-06,0.44,48,39,True
6,1948-01-07,0.41,50,40,True
7,1948-01-08,0.04,48,35,True
8,1948-01-09,0.12,50,31,True
9,1948-01-10,0.74,43,34,True


In [48]:
# Elimino los na pq solo hay 3.
data = data.dropna()

# Quitamos los duplicados
data = data.drop_duplicates()

# data.info()

In [49]:
temp_max = data["TMAX"].max()

print('La temperatura máxima encontrada es', temp_max,'entendemos que estamos en Fº y queremos trabajar en Cº.')

La temperatura máxima encontrada es 103 entendemos que estamos en Fº y queremos trabajar en Cº.


In [50]:
data["TMIN"] = (data.TMIN - 32)*(5/9)

data["TMAX"] = (data.TMAX - 32)*(5/9)

temp_max = data["TMAX"].max().round(2)

temp_min = data["TMIN"].min().round(2)

print('La temperatura máxima en grados Celsius es', temp_max,'Cº.')

La temperatura máxima en grados Celsius es 39.44 Cº.


In [51]:
data.head(40)

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,10.555556,5.555556,True
1,1948-01-02,0.59,7.222222,2.222222,True
2,1948-01-03,0.42,7.222222,1.666667,True
3,1948-01-04,0.31,7.222222,1.111111,True
4,1948-01-05,0.17,7.222222,0.000000,True
5,1948-01-06,0.44,8.888889,3.888889,True
6,1948-01-07,0.41,10.000000,4.444444,True
7,1948-01-08,0.04,8.888889,1.666667,True
8,1948-01-09,0.12,10.000000,-0.555556,True
9,1948-01-10,0.74,6.111111,1.111111,True


In [52]:
conditions = [
          (data['TMIN'] < 10),
          ((data['TMIN'] >= 10) & (data['TMIN'] <= 20)),
          (data['TMIN'] > 20)
             ]

#define results
results = ['Baja', 'Media', 'Alta']

#create new column based on conditions in column1 and column2
data['TMIN'] = np.select(conditions, results)

In [53]:
conditions = [
          (data['TMAX'] < 10),
          ((data['TMAX'] >= 10) & (data['TMAX'] <= 20)),
          (data['TMAX'] > 20)
             ]

#define results
results = ['Baja', 'Media', 'Alta']

#create new column based on conditions in column1 and column2
data['TMAX'] = np.select(conditions, results)

In [54]:
# data.value_counts(data['TMIN'])

data.value_counts(data['TMAX'])

TMAX
Media    12856
Alta      6618
Baja      6074
dtype: int64

In [55]:
conditions = [
          (data['PRCP'] < 1),
          ((data['PRCP'] >= 1) & (data['PRCP'] <= 2)),
          (data['PRCP'] > 2)
             ]

#define results
results = ['Poco', 'Normal', 'Mucho']

#create new column based on conditions in column1 and column2
data['PRCP'] = np.select(conditions, results)

data.head(20)

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,Poco,Media,Baja,True
1,1948-01-02,Poco,Baja,Baja,True
2,1948-01-03,Poco,Baja,Baja,True
3,1948-01-04,Poco,Baja,Baja,True
4,1948-01-05,Poco,Baja,Baja,True
5,1948-01-06,Poco,Baja,Baja,True
6,1948-01-07,Poco,Media,Baja,True
7,1948-01-08,Poco,Baja,Baja,True
8,1948-01-09,Poco,Media,Baja,True
9,1948-01-10,Poco,Baja,Baja,True


In [56]:
data.value_counts(data['TMAX'])

TMAX
Media    12856
Alta      6618
Baja      6074
dtype: int64

Vamos a contar cada una de las distribuciones de clases para los diferentes atributs.

CAMBIAAAAAAR: Hay que ver como clsificar en función de si es True or False (NO a mano)

In [57]:
# Contamos la frecuencia de los valores en la columna de clase 'RAIN'
frecuencias = data['RAIN'].value_counts()
frecuencias.to_frame()

# Guardamos la frecuencia del valor 'True' y 'False' en variables
total_false = frecuencias[0]
total_true = frecuencias[1]

In [58]:
# Contamos la frecuencia de los valores en la columna de clase 'PRCP'
frecuencias = data['PRCP'].value_counts()
print(frecuencias.to_frame())

# Guardamos la frecuencia de los valores 'Poco', 'Normal' y 'Mucho' en variables
total_p_PRCP = frecuencias[0]
total_n_PRCP = frecuencias[1]
total_m_PRCP = frecuencias[2]

print(total_b_PRCP)

         PRCP
Poco    25189
Normal    327
Mucho      32
25189


In [59]:
# Contamos la frecuencia de los valores en la columna de clase 'TMAX'
frecuencias = data['TMAX'].value_counts()
print(frecuencias.to_frame())

# Guardamos la frecuencia de los valores 'Baja', 'Media' y 'Alta' en variables
total_m_TMAX = frecuencias[0]
total_a_TMAX = frecuencias[1]
total_b_TMAX = frecuencias[2]

print(total_b_TMAX)

        TMAX
Media  12856
Alta    6618
Baja    6074
6074


In [60]:
# Contamos la frecuencia de los valores en la columna de clase 'TMAX'
frecuencias = data['TMIN'].value_counts()
print(frecuencias.to_frame())

# Guardamos la frecuencia de los valores 'Baja', 'Media' y 'Alta' en variables
total_b_TMIN = frecuencias[0]
total_m_TMIN = frecuencias[1]
total_a_TMIN = frecuencias[2]

print(total_b_TMIN)

        TMIN
Baja   17000
Media   8543
Alta       5
17000


In [66]:
data_test = pd.DataFrame({'Date': ['15-12-2017', '16-12-2017', '17-12-2017'], 'PRCP': ['Poco', 'Poco', 'Normal'], 'TMAX': ['Alta', 'Media', 'Media'], 'TMIN': ['Media', 'Media', 'Baja']})

data_test.dtypes

Date    object
PRCP    object
TMAX    object
TMIN    object
dtype: object

In [61]:
#split dataset in features and target variable
feature_cols = ['TMAX', 'TMIN', 'PRCP']
X = data[feature_cols] # Features
y = data.RAIN # Target variable

In [62]:
# Split dataset into training set and test set in 75% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [63]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

ValueError: ignored

In [ ]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))